In [2]:
import warnings
warnings.simplefilter("ignore")

In [3]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [4]:
rtc_data = gpd.read_file("../output/rtc_data.geojson")
osm_places = gpd.read_file('../output/osm_places.geojson')
lieuxpublics = gpd.read_file('../input/DonneesVilleQC/vdq-lieupublic.geojson') 

# RTC

In [5]:
rtc_data.head()

,route_id,direction_id,trip_headsign,stop_id,stop_name,stop_lat,stop_lon,stop_sequence,Parcours,Type,geometry
0,1-76,1,Aéroport international Jean-Lesage (Nord),1-1898,Gare de train,46.754501,-71.297424,1,76,Régulier,POINT (-71.29742 46.7545)
1,1-76,1,Aéroport international Jean-Lesage (Nord),1-3515,St-Louis/3515,46.756157,-71.296989,2,76,Régulier,POINT (-71.29699 46.75616)
2,1-76,1,Aéroport international Jean-Lesage (Nord),1-1926,des Hôtels,46.756859,-71.293365,3,76,Régulier,POINT (-71.29336 46.75686)
3,1-76,1,Aéroport international Jean-Lesage (Nord),1-1927,de Valmont/1927,46.757275,-71.290695,4,76,Régulier,POINT (-71.2907 46.75728)
4,1-76,1,Aéroport international Jean-Lesage (Nord),1-1928,W.-Stuart,46.759113,-71.287292,5,76,Régulier,POINT (-71.28729 46.75911)


# Nettoyage données OSM

In [6]:
osm_places = osm_places[~osm_places['amenity'].isin(['parking', 'bus_station'])] # je decide de ne pas les garder

In [7]:
# Fusion name et goepy_name (en priorisant name), si je n'ai pas de valeurs manquantes dans la colonne name j attribut cette valeur a geopy_name 
osm_places.loc[osm_places['name'].notna(), 'geopy_name'] = osm_places['name']

In [8]:
osm_places[osm_places.amenity == 'post_office'].name.unique()

array(['Bureau de poste', None, 'Succursale Poste-Canada',
       'Bureau de Poste Galeries de la Capitale', 'Succursale Postale',
       'Sucursale Postale', 'Bureau de Poste Cap Rouge', 'Postes Canada',
       'UPS', 'Comptoir postal Pharmaprix', 'Brunet - Bureau de poste',
       'DHL', 'Purolator'], dtype=object)

In [9]:
osm_places.head()

,element,id,name,amenity,Category,adresse_geopy,geopy_name,geometry
0,node,49474611,Caisse Populaire Desjardins,bank,Financial,"Caisse Populaire Desjardins, Rue de l'Universi...",Caisse Populaire Desjardins,POINT (-71.26978 46.7792)
1,node,265847671,La Roulotte du Boulevard,fast_food,Food,"La Roulotte du Boulevard, Avenue du Bourg-Roya...",La Roulotte du Boulevard,POINT (-71.21603 46.85481)
2,node,275074044,Petro-Canada,fuel,Transport,"Petro-Canada, Boulevard du Lac, Notre-Dame-des...",Petro-Canada,POINT (-71.31924 46.91669)
3,node,275074469,EKO,fuel,Transport,"EKO, 65, Boulevard Pierre-Bertrand, Saint-Jose...",EKO,POINT (-71.29489 46.84882)
5,node,277667516,Irving,fuel,Transport,"Irving, 750, Boulevard Charest Ouest, Saint-Sa...",Irving,POINT (-71.24392 46.80468)


In [10]:
mots_cles_adress = r'^(\d+|Rue|Avenue|Boulevard|Rang|Chemin|Route|Place|Autoroute)'

adresse = osm_places['geopy_name'].str.contains(mots_cles_adress, case=False, regex=True, na=False) # je verifie si le nom ressemble plus a une adresse qu a un nom de place 
nom_valide = osm_places['name'].notna() # si j ai un nom valide dans la colonne name et pas une valeur manquante

# Filtre final, on garde si (c'est un nom de place) OU SI (ce n'est pas une adresse)
place_to_keep = nom_valide | ~ adresse
osm_places = osm_places[place_to_keep]

In [11]:
pd.DataFrame(osm_places.groupby(['amenity']).size())

,0
amenity,
atm,23
bank,85
bar,41
bicycle_rental,163
cafe,142
clinic,30
college,14
community_centre,56
dentist,26


# Fusion OSM et données villde de QC ?

In [12]:
# Projection WGS84 vers le NAD83 MTM Zone 7
osm_places_m = osm_places.to_crs(epsg=32198)
lieuxpublics_m = lieuxpublics.to_crs(epsg=32198)

# Jointure spatiale dans un rayon de 10m : si deux points sont a moins de 10m, cela veut peut etre dire qu'il s'agit de la meme place  
df_merged = gpd.sjoin_nearest(
    osm_places_m, 
    lieuxpublics_m, 
    how="inner",        
    max_distance=10,    
    distance_col="distance_metres"
)
df_merged[['geopy_name', 'NOM_TOPOGRAPHIE', 'distance_metres']].head(30)

,geopy_name,NOM_TOPOGRAPHIE,distance_metres
33,Centre du Nouvel-Horizon,École spécialisée du Nouvel Horizon,7.322880
50,Les Loisirs Montcalm,Centre de loisirs Montcalm,2.586349
92,Centre culturel Georges Dor,Centre culturel Georges-Dor,3.788206
92,Centre culturel Georges Dor,Bureau d'arrondissement de La Haute-Saint-Charles,3.788206
119,Bibliothèque Lebourgneuf,Centre communautaire Lebourgneuf,2.468229
136,Bureau d'arrondissement de Beauport,Stationnement Centre administratif Rainville,6.598937
156,Bibliothèque Collège-Des-Jésuites,Bibliothèque Collège-des-Jésuites,6.830225
157,Bibliothèque de Neufchâtel,Bibliothèque Neufchâtel,0.628762
438,Centre récréatif de Saint-Rodrigue,Centre récréatif Saint-Rodrigue,2.161596
444,La Sablière,Centre de loisirs la Sablière,4.604477


In [ ]:
# Il y a bel et bien des doublons Centre culturel Georges Dor" et "Centre culturel Georges-Dor" , il s agit du meme batiment 
# Il y a aussi des mauvais match, "Centre culturel Georges Dor" et "Bureau d'arrondissement..." c'est juste un bureau situé dans le même bâtiment ou un batiment juste à côté

# Pistes / idées a tester
# Utiliser la comparaison textuelle pour supprimer les doublons dans les jeux de données en plus de la distance 
# Creer un score de similarité des textes et fusionner avec les distances pour decider s'il s'agit de la mme place ou non 
# Ce code est une ébauche, donc cette approche est à explorer plus tard

In [14]:
osm_places.to_file("../output/osm_places_v2.geojson", driver='GeoJSON')